<a href="https://colab.research.google.com/github/NganTran-0017/HIDS/blob/main/HIDS_Data_Processing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Update remove duplicate between train and test --> create independent test
check why test set after removing the intersection (aka independent test) has no frequent record? --> no, set(independent test) has fewer records than independent test.
So the previous clean_data method removes duplication and overlap instances, while the new method (remove_duplicate) only removes overlap instance


In [25]:
#@title Specify parameters before running


SZ =  1#@param {type:"number"}         # Indicate a fraction number to sample train set when it's too big. Located in Data Partition

SEQ_WINDOW =  25#@param {type:"integer"} # Indicate the window length to parse the sequence into. Used in Data Parsing section

BATCH_SZ =  32#@param {type:"integer"} # Indicate the window length to parse the sequence into. Used in Data Parsing section

EPOCHS =  2#@param {type:"integer"} # Indicate the window length to parse the sequence into. Used in Data Parsing section

# Indicate to clean data or not. Used in Data Cleaning section
CLEAN = True #@param {type:"boolean"}


In [26]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from scipy import stats
from sklearn import metrics
from sklearn.metrics import confusion_matrix, classification_report, roc_auc_score, accuracy_score, roc_curve, auc, recall_score, precision_score,plot_confusion_matrix
from sklearn.model_selection import train_test_split
%matplotlib inline

import nltk
nltk.download("popular")

## Tokenizing syscall sequences into n-grams of 6

from nltk.tokenize import word_tokenize
from nltk import ngrams

[nltk_data] Downloading collection 'popular'
[nltk_data]    | 
[nltk_data]    | Downloading package cmudict to /root/nltk_data...
[nltk_data]    |   Package cmudict is already up-to-date!
[nltk_data]    | Downloading package gazetteers to /root/nltk_data...
[nltk_data]    |   Package gazetteers is already up-to-date!
[nltk_data]    | Downloading package genesis to /root/nltk_data...
[nltk_data]    |   Package genesis is already up-to-date!
[nltk_data]    | Downloading package gutenberg to /root/nltk_data...
[nltk_data]    |   Package gutenberg is already up-to-date!
[nltk_data]    | Downloading package inaugural to /root/nltk_data...
[nltk_data]    |   Package inaugural is already up-to-date!
[nltk_data]    | Downloading package movie_reviews to
[nltk_data]    |     /root/nltk_data...
[nltk_data]    |   Package movie_reviews is already up-to-date!
[nltk_data]    | Downloading package names to /root/nltk_data...
[nltk_data]    |   Package names is already up-to-date!
[nltk_data]    | Do

#**Processing data**

Use the given datasets in our GitHub to load the data

## **ADFA-LD**

In [ ]:
# Load ADFA-LD and unzip them
! wget 'https://github.com/NganTran-0017/HIDS/blob/main/Datasets/ADFA-LD/Training_Data_Master.zip?raw=true'
! wget 'https://github.com/NganTran-0017/HIDS/blob/main/Datasets/ADFA-LD/Attack_Data_Master.zip?raw=true'
# Rename files
! mv Attack_Data_Master.zip?raw=true   Attack_Data_Master.zip
! mv Training_Data_Master.zip?raw=true Training_Data_Master.zip

# Unzip files
! unzip Training_Data_Master.zip
! unzip Attack_Data_Master.zip

from glob import glob
import os
train_dir = 'Training_Data_Master/'
test_dir  = 'Attack_Data_Master/'

# Read data in a dataframe
def read_in_data(path, is_normal = 1): 
  lines = []
  if is_normal == 1:
    for filename in os.listdir(path): # iterate through all label folders in a directory 
      with open(path+filename) as f:
        lines.append(f.read())
       # data = pd.read_csv(path+filename, sep=' ', header=None)
       # list_of_dataframes.append(data.stack())
    
  else:
    for folder in os.listdir(path): # iterate through different attack folders in a directory
      files = glob(str(path + folder +"/*.txt"), recursive=False) # get a list of files from each label folder 
      for filename in files: 
        with open(filename) as f:
          lines.append(f.read())
        #data = pd.read_csv(filename, sep=' ', header=None)
        #lines.append(data.stack())

  df = pd.DataFrame(lines)
  return df   


# Trainning set
df = read_in_data(train_dir)
df.rename(columns={0:'Syscall Sequence'}, inplace=True)

print('Normal training data size:', df.shape)
print(df.head(5))

# Testing set
intrusiondf = read_in_data(test_dir, is_normal = 0)
intrusiondf.rename(columns={0:'Syscall Sequence'}, inplace=True)

print('Intrusion Testing data size:', intrusiondf.shape)
intrusiondf.head(5)

## **UNM and MIT Lpr**

In [28]:
  ## Uncomment each line to load Normal data

# Synthetic sendmail csv_file = ['bounce-1.int', 'bounce.int', 'bounce-2.int', 'plus.int', 'queue.int', 'sendmail.daemon.int', 'sendmail.log.int']; DATA = 'Synthetic Sendmail'; DATA_I='Synthetic_Sendmail'
#!wget 'https://raw.githubusercontent.com/NganTran-0017/HIDS/main/Datasets/UNM/1.Synthetic%20Sendmail/normal-data/bounce-1.int'
#!wget 'https://raw.githubusercontent.com/NganTran-0017/HIDS/main/Datasets/UNM/1.Synthetic%20Sendmail/normal-data/bounce.int'
#!wget 'https://raw.githubusercontent.com/NganTran-0017/HIDS/main/Datasets/UNM/1.Synthetic%20Sendmail/normal-data/bounce-2.int'
#!wget 'https://raw.githubusercontent.com/NganTran-0017/HIDS/main/Datasets/UNM/1.Synthetic%20Sendmail/normal-data/plus.int'
#!wget 'https://raw.githubusercontent.com/NganTran-0017/HIDS/main/Datasets/UNM/1.Synthetic%20Sendmail/normal-data/queue.int'
#!wget 'https://raw.githubusercontent.com/NganTran-0017/HIDS/main/Datasets/UNM/1.Synthetic%20Sendmail/normal-data/sendmail.daemon.int'
#!wget 'https://raw.githubusercontent.com/NganTran-0017/HIDS/main/Datasets/UNM/1.Synthetic%20Sendmail/normal-data/sendmail.log.int'

# LIVE LPR csv_file = ['lpr-normal-10.txt', 'lpr-normal-11.txt']; DATA = 'Live Lpr'; DATA_I='Live-Lpr' 
#!wget 'https://raw.githubusercontent.com/NganTran-0017/HIDS/main/Datasets/UNM/4.%20Live%20lpr/Normal/real/lpr-normal-11.txt'
#!wget 'https://raw.githubusercontent.com/NganTran-0017/HIDS/main/Datasets/UNM/4.%20Live%20lpr/Normal/real/lpr-normal-10.txt'

# MIT live lpr csv_file = [ 'mit-lpr-mar.txt']; DATA = 'MIT Live Lpr'; DATA_I='MIT-Lpr' 
#!wget 'https://raw.githubusercontent.com/NganTran-0017/HIDS/main/Datasets/MIT/mit-lpr-mar.txt'

# LOGIN and PS csv_file = [ 'login-normal.txt', 'ps-normal.txt']; DATA = 'Login and Ps'; DATA_I ="Login-and-Ps"
#!wget 'https://raw.githubusercontent.com/NganTran-0017/HIDS/main/Datasets/UNM/7.Login_and_ps/normal/login-normal.txt'
#!wget 'https://raw.githubusercontent.com/NganTran-0017/HIDS/main/Datasets/UNM/7.Login_and_ps/normal/ps-normal.txt'

# INETD csv_file = [ 'inetd-live-unm.int']; DATA = 'Inetd';DATA_I ="Inetd" 
#!wget 'https://raw.githubusercontent.com/NganTran-0017/HIDS/main/Datasets/UNM/8.Inetd/inetd-live-unm.int'

# STIDE csv_file = [ 'stide-normal-500k.txt']; DATA = 'Stide';DATA_I ='Stide' 
#!wget 'https://raw.githubusercontent.com/NganTran-0017/HIDS/main/Datasets/UNM/10.Stide/stide-normal-500k.txt'

# Live Named  csv_file = [ 'normal-named-live-2k.txt']; DATA = 'Live Named';DATA_I ='Live-Named' 
#!wget 'https://raw.githubusercontent.com/NganTran-0017/HIDS/main/Datasets/UNM/6.Live_named/normal-named-live-2k.txt'

# Xlock 
csv_file = [ 'normal-xlock.txt']; DATA = 'Xlock';DATA_I='Xlock' 
!wget 'https://raw.githubusercontent.com/NganTran-0017/HIDS/main/Datasets/UNM/5.xlock/normal-xlock.txt'

# Synthetic Ftp 
#csv_file = [ 'nonself1.int','nonself2.int']; DATA = 'Synthetic Ftp'; DATA_I='Synthetic-Ftp'
#!wget 'https://raw.githubusercontent.com/NganTran-0017/HIDS/main/Datasets/UNM/2.Synthetic%20Ftp/nonself1.int'
#!wget 'https://raw.githubusercontent.com/NganTran-0017/HIDS/main/Datasets/UNM/2.Synthetic%20Ftp/nonself2.int'

# Synthetic lpr csv_file = ['syn.int']; DATA = 'Synthetic Lpr';DATA_I='Synthetic-Lpr'
#!wget 'https://raw.githubusercontent.com/NganTran-0017/HIDS/main/Datasets/UNM/3.Synthetic-lpr/syn.int'




# Concat a list of files into normal df
list_of_dataframes = []
for filename in csv_file:
    list_of_dataframes.append(pd.read_csv(filename, sep=' ', header=None, engine='python'))
df = pd.concat(list_of_dataframes)

# Check number of columns, if > 2, then drop the excess
if len(df.columns) > 2:
    df=df.drop(labels=None, axis=1, columns = [2,3])
df =df.rename(columns= {0:"PID", 1:"Syscall"})

print('Normal data size:', df.shape)


  ## Uncomment each line to load Intrusion data:

# Synthetic sendmail csv_file = ['sm-10763.int', 'fwd-loops-1.int', 'fwd-loops-2.int', 'fwd-loops-3.int', 'fwd-loops-4.int', 'fwd-loops-5.int','sm-280.int', 'sm-314.int','sm-10801.int', 'sm-10814.int']
#!wget 'https://raw.githubusercontent.com/NganTran-0017/HIDS/main/Datasets/UNM/1.Synthetic%20Sendmail/Intrusion-trace-data/fwd-loops-1.int'
#!wget 'https://raw.githubusercontent.com/NganTran-0017/HIDS/main/Datasets/UNM/1.Synthetic%20Sendmail/Intrusion-trace-data/fwd-loops-2.int'
#!wget 'https://raw.githubusercontent.com/NganTran-0017/HIDS/main/Datasets/UNM/1.Synthetic%20Sendmail/Intrusion-trace-data/fwd-loops-3.int'
#!wget 'https://raw.githubusercontent.com/NganTran-0017/HIDS/main/Datasets/UNM/1.Synthetic%20Sendmail/Intrusion-trace-data/fwd-loops-4.int'
#!wget 'https://raw.githubusercontent.com/NganTran-0017/HIDS/main/Datasets/UNM/1.Synthetic%20Sendmail/Intrusion-trace-data/fwd-loops-5.int'
#!wget 'https://raw.githubusercontent.com/NganTran-0017/HIDS/main/Datasets/UNM/1.Synthetic%20Sendmail/Intrusion-trace-data/sm-10763.int'
#!wget 'https://raw.githubusercontent.com/NganTran-0017/HIDS/main/Datasets/UNM/1.Synthetic%20Sendmail/Intrusion-trace-data/sm-280.int'
#!wget 'https://raw.githubusercontent.com/NganTran-0017/HIDS/main/Datasets/UNM/1.Synthetic%20Sendmail/Intrusion-trace-data/sm-314.int'
#!wget 'https://raw.githubusercontent.com/NganTran-0017/HIDS/main/Datasets/UNM/1.Synthetic%20Sendmail/Intrusion-trace-data/sm-10801.int'
#!wget 'https://raw.githubusercontent.com/NganTran-0017/HIDS/main/Datasets/UNM/1.Synthetic%20Sendmail/Intrusion-trace-data/sm-10814.int'

# LIVE LPR csv_file =['exploit-unm.int'] 
#!wget 'https://raw.githubusercontent.com/NganTran-0017/HIDS/main/Datasets/UNM/4.%20Live%20lpr/Intrusion/exploit-unm.int'

# MIT live lpr csv_file = [ 'exploit-ai.int'] 
#!wget 'https://raw.githubusercontent.com/NganTran-0017/HIDS/main/Datasets/MIT/exploit-ai.int'

# LOGIN and PS csv_file = [ 'login-homegrown.int','ps-homegrown.int','login-recovered.int','ps-recovered.int']
#!wget 'https://raw.githubusercontent.com/NganTran-0017/HIDS/main/Datasets/UNM/7.Login_and_ps/intrusion/ps-recovered.int'
#!wget 'https://raw.githubusercontent.com/NganTran-0017/HIDS/main/Datasets/UNM/7.Login_and_ps/intrusion/ps-homegrown.int'
#!wget 'https://raw.githubusercontent.com/NganTran-0017/HIDS/main/Datasets/UNM/7.Login_and_ps/intrusion/login-recovered.int'
#!wget 'https://raw.githubusercontent.com/NganTran-0017/HIDS/main/Datasets/UNM/7.Login_and_ps/intrusion/login-homegrown.int'

# INETD csv_file = [ 'inetd-intrusion.int'] 
#!wget 'https://raw.githubusercontent.com/NganTran-0017/HIDS/main/Datasets/UNM/8.Inetd/intrusion/inetd-intrusion.int'

# STIDE csv_file = [ 'stide-intrusion'] 
#!wget 'https://raw.githubusercontent.com/NganTran-0017/HIDS/main/Datasets/UNM/10.Stide/intrusion/stide-intrusion'

# Live Named  csv_file = [ 'exploit-1.int','exploit-2.int'] 
#!wget 'https://raw.githubusercontent.com/NganTran-0017/HIDS/main/Datasets/UNM/6.Live_named/intrusion/exploit-1.int'
#!wget 'https://raw.githubusercontent.com/NganTran-0017/HIDS/main/Datasets/UNM/6.Live_named/intrusion/exploit-2.int'

# Xlock 
csv_file = [ 'nonself.cs.unm.edu-07.24.97-xlock-2822_new.log.int', 'nonself.cs.unm.edu-07.25.97-xlock-2691_new.log.int']
!wget 'https://raw.githubusercontent.com/NganTran-0017/HIDS/main/Datasets/UNM/5.xlock/intrusion/nonself.cs.unm.edu-07.25.97-xlock-2691_new.log.int'
!wget 'https://raw.githubusercontent.com/NganTran-0017/HIDS/main/Datasets/UNM/5.xlock/intrusion/nonself.cs.unm.edu-07.24.97-xlock-2822_new.log.int'

# Synthetic Ftp csv_file = [ 'exploit2.int']
#!wget 'https://raw.githubusercontent.com/NganTran-0017/HIDS/main/Datasets/UNM/2.Synthetic%20Ftp/intrusion/exploit2.int'

# Synthetic Lpr csv_file = [ 'exploit-unm.int']
#!wget 'https://raw.githubusercontent.com/NganTran-0017/HIDS/main/Datasets/UNM/3.Synthetic-lpr/intrusion/exploit-unm.int'

list_of_dataframes = []
for filename in csv_file:
    list_of_dataframes.append(pd.read_csv(filename, sep=' ', header=None, engine='python'))
intrusiondf = pd.concat(list_of_dataframes)

if len(intrusiondf.columns) > 2:
    intrusiondf = intrusiondf.drop(labels=None, axis=1, columns = [2,3])
intrusiondf = intrusiondf.rename(columns= {0:"PID", 1:"Syscall"})

print('intrusion data size: ', intrusiondf.shape)

print('Normal:',df.head(3))
#print(df['PID'].value_counts())
print('Intrusion:',intrusiondf.head())

--2022-02-20 03:13:55--  https://raw.githubusercontent.com/NganTran-0017/HIDS/main/Datasets/UNM/5.xlock/normal-xlock.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 192277 (188K) [text/plain]
Saving to: ‘normal-xlock.txt.1’

normal-xlock.txt.1  100%[===================>] 187.77K  --.-KB/s    in 0.004s  

2022-02-20 03:13:55 (45.9 MB/s) - ‘normal-xlock.txt.1’ saved [192277/192277]

Normal data size: (25225, 2)
--2022-02-20 03:13:55--  https://raw.githubusercontent.com/NganTran-0017/HIDS/main/Datasets/UNM/5.xlock/intrusion/nonself.cs.unm.edu-07.25.97-xlock-2691_new.log.int
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.c

**Change to covert all syscall of 1 PID into a data record. Pasrse each data record to a length of 10 or 15, clean frequent records.**

In [29]:
intrusiondf

,PID,Syscall
0,2826,5
1,2826,90
2,2826,6
3,2826,125
4,2826,125
...,...,...
455,2695,126
456,2695,54
457,2695,126
458,2695,57


**Create syscall sequence per pid**

In [30]:
# This function groups data by PID, so the sequences appear by PID instead of by order, in case it was interrupted by other PID
## It returns a dict with PID as key and syscall seq as item
def group_syscalls_by_pid (data):
  seq_per_pid = {}
  for p in data['PID'].unique():
    filt = data['PID'] == p
    seq = data.loc[filt]['Syscall'].values.astype(str)
    seq_per_pid[p] = ' '.join(seq)
  return seq_per_pid

if df.shape[1] == 2:
  print('Number of unique PID in normal data:', len(df['PID'].value_counts()))
  print('Number of unique PID in intrusion data:', len(intrusiondf['PID'].value_counts()))

  # Group normal df by PID and drop PID column
  normal_seq_per_pid = group_syscalls_by_pid(df)
  print('Number of unique PID in normal:', len(normal_seq_per_pid))
  #print('Normal PIDs and its sequences: ',normal_seq_per_pid)

  # Do the same thing to intrusion PID
  intrusion_seq_per_pid = group_syscalls_by_pid(intrusiondf)
  print('Number of unique PID in intrusion:', len(intrusion_seq_per_pid))
  #print('Intrusion PIDs and its sequences: ', intrusion_seq_per_pid)

else:
  print('ADFA-LD data does not have PID column')

Number of unique PID in normal data: 20
Number of unique PID in intrusion data: 2
Number of unique PID in normal: 20
Number of unique PID in intrusion: 2


In [31]:
# Drop a sequence if its total len is less than 3
def remove_small_seq(pid_seq_dict):
  removed_pid = []
  for pid in pid_seq_dict:
    seq_list = pid_seq_dict[pid].split()
    if len(seq_list) < 3:
      print('Remove PID %d which only has %d syscals in its sequence: %s' % (pid, len(seq_list), pid_seq_dict[pid]))
      removed_pid.append(pid)

  [pid_seq_dict.pop(pid) for pid in removed_pid]
  return pid_seq_dict

# Clean small intrusion sequences
print('Clean small normal seq: \nNum PID in Normal before:', len(normal_seq_per_pid))
normal_seq_per_pid = remove_small_seq(normal_seq_per_pid)
df = pd.DataFrame.from_dict(normal_seq_per_pid, orient = 'index', columns=['Syscall Sequence'] )
print('Num PID in Normal after:', len(normal_seq_per_pid))

# Clean small intrusion sequences
print('\n\nClean small intrusion seq: \nNum PID in Intrusion before:', len(intrusion_seq_per_pid))
intrusion_seq_per_pid = remove_small_seq(intrusion_seq_per_pid)
intrusiondf = pd.DataFrame.from_dict(intrusion_seq_per_pid, orient = 'index', columns=['Syscall Sequence'] )
print('Num PID in Intrusion after:', len(intrusion_seq_per_pid))

Clean small normal seq: 
Num PID in Normal before: 20
Num PID in Normal after: 20


Clean small intrusion seq: 
Num PID in Intrusion before: 2
Num PID in Intrusion after: 2


## **Data Parsing**

In [32]:
df

,Syscall Sequence
1877,5 90 6 125 125 24 49 47 50 106 5 90 6 5 5 3 90...
1883,5 90 6 125 125 24 49 47 50 106 5 90 6 5 5 3 90...
1889,5 90 6 125 125 24 49 47 50 106 5 90 6 5 5 3 90...
1895,5 90 6 125 125 24 49 47 50 106 5 90 6 5 5 3 90...
1903,5 90 6 125 125 24 49 47 50 106 5 90 6 5 5 3 90...
1909,5 90 6 125 125 24 49 47 50 106 5 90 6 5 5 3 90...
1915,5 90 6 125 125 24 49 47 50 106 5 90 6 5 5 3 90...
1921,5 90 6 125 125 24 49 47 50 106 5 90 6 5 5 3 90...
1929,5 90 6 125 125 24 49 47 50 106 5 90 6 5 5 3 90...
1935,5 90 6 125 125 24 49 47 50 106 5 90 6 5 5 3 90...


In [33]:
## Parse an entire Syscall seq per PID into smaller sequences of size 15
def parse_seq(seq_per_pid):
  sequences = pd.DataFrame()
  for _, row in seq_per_pid.iterrows():
    token = word_tokenize(row['Syscall Sequence'])  # Tokenize the string of sequence
 
    # Parse the sequence into length of 15
    sequences=sequences.append(list(nltk.ngrams(token, SEQ_WINDOW, pad_right=True, right_pad_symbol=-1)))
    #print('PID %d - seq len: %d'% (p, len(sequences)))
  return sequences

#normal = parse_seq(normal_seq_per_pid)
normal = parse_seq(df)
normal

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24
0,5,90,6,125,125,24,49,47,50,106,5,90,6,5,5,3,90,90,90,90,6,6,125,5,5
1,90,6,125,125,24,49,47,50,106,5,90,6,5,5,3,90,90,90,90,6,6,125,5,5,3
2,6,125,125,24,49,47,50,106,5,90,6,5,5,3,90,90,90,90,6,6,125,5,5,3,90
3,125,125,24,49,47,50,106,5,90,6,5,5,3,90,90,90,90,6,6,125,5,5,3,90,90
4,125,24,49,47,50,106,5,90,6,5,5,3,90,90,90,90,6,6,125,5,5,3,90,90,90
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
607,4,3,163,6,1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1
608,3,163,6,1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1
609,163,6,1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1
610,6,1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1


In [34]:
print('Parsing Intrusion')
intrusion = parse_seq(intrusiondf)
intrusion

Parsing Intrusion


,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24
0,5,90,6,125,125,24,49,47,50,106,5,90,6,5,5,3,90,90,90,90,6,6,125,5,5
1,90,6,125,125,24,49,47,50,106,5,90,6,5,5,3,90,90,90,90,6,6,125,5,5,3
2,6,125,125,24,49,47,50,106,5,90,6,5,5,3,90,90,90,90,6,6,125,5,5,3,90
3,125,125,24,49,47,50,106,5,90,6,5,5,3,90,90,90,90,6,6,125,5,5,3,90,90
4,125,24,49,47,50,106,5,90,6,5,5,3,90,90,90,90,6,6,125,5,5,3,90,90,90
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
455,126,54,126,57,1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1
456,54,126,57,1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1
457,126,57,1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1
458,57,1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1


Start tokenizing system calls into n-grams as specified in the params at the top of the page.

## **Preparing data for data partition**
- Combine data and labels together. 
- Combine data from each class together --> Create a pool of data 
- Split data pool into train and test

## **Cleaning**
- Remove dupplication between train and test

- 2 directions for Testing set:
  *  Test with clean data: Remove overlap and dupplication between Normal and Intrusion in Test data
  *  Test with Unclean data: Leave Test data as is

- 2 directions Training set:
  *   Clean model: Remove overlap and dupplication between Normal and Intrusion in Train data
  *   Unclean model: Leave Train data as is



**Add Label to Data**

In [35]:
# Intrusion sequences is labeled 1.
intrusion['Label'] = 1
intrusion.head(5)

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,Label
0,5,90,6,125,125,24,49,47,50,106,5,90,6,5,5,3,90,90,90,90,6,6,125,5,5,1
1,90,6,125,125,24,49,47,50,106,5,90,6,5,5,3,90,90,90,90,6,6,125,5,5,3,1
2,6,125,125,24,49,47,50,106,5,90,6,5,5,3,90,90,90,90,6,6,125,5,5,3,90,1
3,125,125,24,49,47,50,106,5,90,6,5,5,3,90,90,90,90,6,6,125,5,5,3,90,90,1
4,125,24,49,47,50,106,5,90,6,5,5,3,90,90,90,90,6,6,125,5,5,3,90,90,90,1


In [36]:
# Normal sequences is labeled 0.
print('Normal len:',len(normal),'\nIntrusion len:', len(intrusion))
normal['Label'] = 0
normal.head(5)

Normal len: 25225 
Intrusion len: 949


,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,Label
0,5,90,6,125,125,24,49,47,50,106,5,90,6,5,5,3,90,90,90,90,6,6,125,5,5,0
1,90,6,125,125,24,49,47,50,106,5,90,6,5,5,3,90,90,90,90,6,6,125,5,5,3,0
2,6,125,125,24,49,47,50,106,5,90,6,5,5,3,90,90,90,90,6,6,125,5,5,3,90,0
3,125,125,24,49,47,50,106,5,90,6,5,5,3,90,90,90,90,6,6,125,5,5,3,90,90,0
4,125,24,49,47,50,106,5,90,6,5,5,3,90,90,90,90,6,6,125,5,5,3,90,90,90,0


## **Partition Training and Testing dataset 70/30**
- Combine both classes together then split data.
- We do not need to bootstrap since there are more than enough data from both classes.


In [55]:
# combine normal(train) and intrusion(test) data and split them into training and testing sets
df = normal.append(intrusion, ignore_index=True).astype(int)
print('Df sz:', df.shape)

# Spliting into training and testing
x_train, x_test, y_train, y_test = train_test_split(df, df['Label'], test_size = 0.30, shuffle=True, stratify=df['Label'])

# Reset index of training and testing sets
x_train.reset_index(drop=True, inplace=True); y_train.reset_index(drop=True, inplace=True)
x_test.reset_index(drop=True, inplace=True);  y_test.reset_index(drop=True, inplace=True)

print('Train sz:',len(x_train), len(y_train))
intrusion_train = y_train.loc[y_train == 1]
normal_train = y_train.loc[y_train == 0]
print('Train set: Intrusion vs. Normal cases', len(y_train.iloc[intrusion_train] ), len(y_train.iloc[normal_train] ))

print('Test sz:', len(x_test), len(y_test))
intrusion_test = y_test.loc[y_test == 1]
normal_test = y_test.loc[y_test == 0]
print('Test set: Intrusion vs. Normal cases', len(y_test.iloc[intrusion_test] ), len(y_test.iloc[normal_test] ))

Df sz: (26174, 26)
Train sz: 18321 18321
Train set: Intrusion vs. Normal cases 664 17657
Test sz: 7853 7853
Test set: Intrusion vs. Normal cases 285 7568


## **Data Cleaning**
**Remove Overlap between Train and Test**
--> Ensures reliability in model performance

**Remove Dupplicate or Frequent Data in Test""
--> Ensures a fair evaluation because if Test has a lot of dupplication, model will either classify all dupplication correctly or incorrectly --> Not fair and not reliable in evaluating model performance 

In [56]:
## Get % of duplicates in both datasets

# Convert normal df to set, and intrusion df to set
def clean_data(normal, intrusion):
  normal_list = normal.values.tolist()
  intrusion_list = intrusion.values.tolist()
  normal_set = set(tuple(i) for i in normal_list)
  intrusion_set = set(tuple(i) for i in intrusion_list)
  print('List sz vs. Set sz of normal sequences: %d vs. %d'% (len(normal_list),len(normal_set)) )
  print('List sz vs. Set sz of intrusion sequences: %d vs. %d'% (len(intrusion_list),len(intrusion_set)) )
    
  normal_dupplication = (len(normal_list) - len(normal_set)) /len(normal_list)*100 
  intrusion_duplication = (len(intrusion_list)-len(intrusion_set))/len(intrusion_list) * 100

  print('Duplication Rate in Normal Class: %.3f%%'% normal_dupplication )
  print('Duplication Rate in Intrusion Class: %.3f%%'% intrusion_duplication) 
 
  c_intrusion = intrusion_set - normal_set 
  overlap_rate =  len(normal_set.intersection(intrusion_set)) / len(normal_set.union(intrusion_set)) * 100
  print('Overlap rate: %.3f%%' % overlap_rate)
  
  #c_normal = normal_set - intrusion_set
  if len(c_intrusion) == 0:
    print(DATA+' No Duplication!')
  if len(c_intrusion) > 0:
    intrusion = pd.DataFrame(c_intrusion)
  else:
    intrusion = pd.DataFrame(intrusion_set)
  #if len(c_normal) > 0:
  #  normal = pd.DataFrame(c_normal)
  #else:
  normal = pd.DataFrame(normal_set)

  print('After cleaning: \nNormal sz:', len(normal), ' Intrusion sz:', len(c_intrusion) )
  return normal, intrusion


def remove_overlap2(train, test):
  train_list = train.values.tolist()
  test_list = test.values.tolist()
  train_set = set(tuple(i) for i in train_list)
  test_set = set(tuple(i) for i in test_list)
  print('List sz vs. Set sz of training sequences: %d vs. %d'% (len(train_list),len(train_set)) )
  print('List sz vs. Set sz of testing sequences: %d vs. %d'% (len(test_list),len(test_set)) )
    
  train_dupplication  = (len(train_list) - len(train_set)) /len(train_list)*100 
  test_duplication    = (len(test_list) - len(test_set))/len(test_list) * 100

  print('Duplication Rate in training set: %.3f%%'% train_dupplication )
  print('Duplication Rate in test set: %.3f%%'% test_duplication) 

  intersect = train_set.intersection(test_set)
  overlap_rate = len(intersect)*100/(len(train_set.union(test_set)))
  print( 'Overlap rate is %.3f%%'% overlap_rate)
  intersection_df = pd.DataFrame.from_dict(intersect).rename(columns={25:'Label'})
  independent_test = pd.merge(intersection_df, test, how = 'outer', indicator = True).query('_merge=="right_only"').drop(columns = '_merge')

  return independent_test

In [57]:
independent_test = remove_overlap2(x_train, x_test)
independent_test

List sz vs. Set sz of training sequences: 18321 vs. 2590
List sz vs. Set sz of testing sequences: 7853 vs. 1581
Duplication Rate in training set: 85.863%
Duplication Rate in test set: 79.868%
Overlap rate is 31.494%


,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,Label
7194,119,4,3,3,3,3,78,4,3,3,3,78,78,54,78,78,104,82,119,4,3,78,82,119,4,0
7195,4,3,78,82,119,4,3,78,82,119,54,3,78,54,78,78,82,4,3,78,82,54,3,78,54,0
7196,119,3,3,3,3,3,3,78,78,78,78,54,78,54,78,78,119,82,54,3,78,54,78,78,82,0
7197,5,3,90,90,90,90,6,6,125,5,5,3,90,90,90,90,6,6,125,91,125,125,125,125,125,1
7198,5,3,90,90,90,90,6,6,125,5,5,3,90,90,90,90,6,6,125,91,125,125,125,125,125,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7848,89,89,89,89,89,89,107,6,45,20,64,24,5,108,45,3,19,6,65,41,55,63,6,54,20,1
7849,3,4,3,82,3,4,3,82,3,3,4,3,82,3,4,3,82,3,4,3,82,3,4,3,82,0
7850,89,89,89,89,89,89,107,6,107,106,5,55,89,89,89,89,89,107,6,107,106,5,55,89,89,1
7851,5,108,45,3,19,6,122,45,45,107,107,107,106,5,55,89,89,89,89,89,89,107,6,107,106,1


intersect = remove_overlap2(x_train, x_test)
intersection_df = pd.DataFrame.from_dict(intersect).rename(columns={25:'Label'})

intersection_df

In [58]:
x_test

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,Label
0,45,45,45,5,106,45,3,6,45,45,45,5,108,45,3,3,3,6,5,108,3,3,3,6,5,0
1,67,67,67,4,4,3,126,54,126,67,67,67,67,4,4,67,106,5,4,6,126,54,126,57,1,1
2,33,5,108,3,3,6,4,55,3,3,4,3,3,5,106,3,6,24,5,108,45,3,19,6,24,0
3,4,3,82,119,4,3,82,119,4,3,82,119,4,3,82,119,4,3,82,119,4,3,82,119,4,0
4,3,82,119,4,3,82,119,4,3,82,119,4,3,82,119,4,3,82,119,4,3,82,119,4,3,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7848,119,4,3,82,119,4,3,82,119,4,3,82,119,4,3,82,119,4,3,82,119,4,3,82,119,0
7849,3,90,90,90,90,6,6,125,5,5,3,90,90,90,90,6,6,125,5,5,3,90,90,90,6,0
7850,6,49,4,3,3,3,3,4,3,4,3,3,4,3,4,3,4,3,4,3,4,3,4,3,4,0
7851,82,119,4,3,82,119,4,3,82,119,4,3,82,119,4,3,82,119,4,3,82,119,4,3,82,0


In [59]:
# Check how much data have left after condensing it as a set
a = independent_test.values.tolist()
a_set = set(tuple(i) for i in a)
len(a_set)

582

left = pd.DataFrame({'ID': [1,2,3,1,4], 'name': ['Ngan', 'Juan', 'Dave', 'Ngan', 'JD']})
right = pd.DataFrame({'ID': [2,2,4], 'name': ['Natti', 'Natti', 'JD']})

pd.merge(left, right, how='outer', indicator=True).query('_merge=="left_only"')

In [60]:
# Split test data in 2 proportions: Clean and Unclean
# Clean: clean test data to test the models
# Unclean: leave test data as is to test the models.

x_test = independent_test.copy()
test_clean = x_test.sample(frac = 0.5)
test_unclean = x_test.drop(test_clean.index)

test_clean

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,Label
7242,6,6,125,5,5,3,90,90,90,90,6,6,125,5,5,3,90,90,90,6,6,125,5,5,5,1
7471,3,3,4,3,4,119,4,3,3,3,3,78,4,3,3,3,78,78,54,78,78,104,82,119,4,0
7695,3,82,119,4,3,82,119,4,3,82,119,4,3,82,119,4,3,82,119,4,3,3,54,78,82,0
7677,126,67,126,54,54,126,67,67,67,4,4,3,126,54,126,67,67,67,67,4,4,67,106,5,4,1
7204,4,3,82,119,4,3,3,3,3,4,3,3,3,4,3,4,119,4,3,3,3,3,78,4,3,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7222,82,54,3,78,54,78,78,82,54,3,78,54,78,78,82,119,4,3,78,82,119,4,3,78,82,0
7489,119,4,3,78,82,119,4,3,78,82,119,54,3,78,54,78,78,82,4,3,78,82,54,3,78,0
7831,4,3,3,78,78,78,78,54,78,78,104,82,54,3,54,78,54,78,78,82,54,3,54,78,54,0
7722,54,78,54,78,78,82,54,3,78,54,78,54,78,78,82,54,3,54,78,54,78,78,82,54,3,0


In [61]:
test_unclean

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,Label
7195,4,3,78,82,119,4,3,78,82,119,54,3,78,54,78,78,82,4,3,78,82,54,3,78,54,0
7197,5,3,90,90,90,90,6,6,125,5,5,3,90,90,90,90,6,6,125,91,125,125,125,125,125,1
7198,5,3,90,90,90,90,6,6,125,5,5,3,90,90,90,90,6,6,125,91,125,125,125,125,125,1
7199,4,3,78,82,119,54,3,4,119,119,119,4,3,119,3,4,4,3,163,119,6,1,-1,-1,-1,0
7200,78,54,78,78,82,119,4,3,78,82,119,4,3,78,82,119,4,3,78,82,119,54,3,4,3,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7837,6,4,55,3,3,4,3,3,5,106,3,6,24,5,108,45,3,19,6,49,4,3,3,3,3,0
7842,163,6,119,1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,0
7846,4,3,3,4,54,78,54,78,78,82,54,3,4,54,78,54,78,78,82,54,3,78,54,78,78,0
7851,5,108,45,3,19,6,122,45,45,107,107,107,106,5,55,89,89,89,89,89,89,107,6,107,106,1


In [62]:
# Separate normal and intrusion in Test Clean so that I can call func clean_data on them

## Filter normal data from Test and drop label column
filt = test_clean.loc[:, 'Label'] == 0
test_clean_normal = test_clean.loc[filt]
test_clean_normal.drop(columns = 'Label', inplace = True)

## Filter Intrusion data from Test and drop label column
test_clean_intrusion = test_clean.loc[~filt]
test_clean_intrusion.drop(columns = 'Label', inplace = True)


/usr/local/lib/python3.7/dist-packages/pandas/core/frame.py:4913: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


In [63]:
test_clean_normal, test_clean_intrusion = clean_data(test_clean_normal, test_clean_intrusion)

List sz vs. Set sz of normal sequences: 276 vs. 269
List sz vs. Set sz of intrusion sequences: 54 vs. 46
Duplication Rate in Normal Class: 2.536%
Duplication Rate in Intrusion Class: 14.815%
Overlap rate: 0.000%
After cleaning: 
Normal sz: 269  Intrusion sz: 46


In [64]:
test_clean_normal

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24
0,3,54,78,104,82,54,3,54,78,82,54,3,4,3,3,3,4,3,4,78,4,3,3,3,3
1,4,119,3,3,3,3,78,4,3,3,3,78,78,54,78,78,104,82,119,4,3,78,82,119,4
2,119,82,54,3,78,54,78,78,82,4,3,78,82,119,54,3,78,54,78,78,82,4,3,78,82
3,153,156,157,122,55,33,5,108,3,3,6,4,55,3,3,4,3,3,5,106,3,6,4,4,4
4,54,3,78,54,78,78,82,54,3,78,54,78,78,82,119,4,3,78,82,54,3,4,4,3,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
264,82,119,4,3,82,3,82,119,4,3,82,3,82,119,4,3,82,3,82,54,3,4,3,82,3
265,4,119,119,78,4,3,3,3,3,78,78,54,78,78,104,82,119,4,3,78,82,119,4,3,78
266,3,3,4,3,4,78,4,3,3,3,3,78,4,3,3,4,3,3,78,78,78,78,54,78,78
267,67,67,4,3,3,126,4,54,3,4,54,78,104,82,54,3,4,3,3,3,4,3,4,78,4


In [65]:
test_clean_intrusion

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24
0,4,3,4,3,4,126,54,126,67,67,67,67,106,106,106,106,106,106,80,126,2,57,126,126,126
1,6,107,106,5,55,89,89,89,89,89,89,89,89,89,89,107,6,107,106,5,55,89,89,89,89
2,125,125,136,45,13,45,45,45,45,45,45,45,45,45,5,108,45,3,3,3,6,5,108,3,3
3,89,89,89,89,89,89,89,89,107,6,107,106,5,55,89,89,89,89,89,89,89,89,107,6,20
4,55,89,89,89,89,89,107,6,107,106,5,55,89,89,89,107,6,107,106,5,55,89,89,89,89
5,5,55,89,89,89,89,89,89,107,6,107,106,5,55,89,89,89,89,89,107,6,107,106,5,55
6,125,125,125,125,125,125,125,125,136,45,13,45,45,45,45,45,45,45,45,45,5,108,45,3,3
7,107,106,5,55,89,89,89,89,89,89,89,89,107,6,20,45,64,24,5,108,45,3,19,6,65
8,5,108,45,3,19,6,106,126,54,54,126,67,67,67,45,4,3,4,3,4,3,4,126,54,126
9,6,6,125,5,5,3,90,90,90,90,6,6,125,5,5,3,90,90,90,6,6,125,5,5,5


In [66]:
# Add label back to data
test_clean_normal['Label'] = 0
test_clean_intrusion['Label'] = 1
test_clean = test_clean_normal.append(test_clean_intrusion, ignore_index= True)

test_clean
## Save Train, Test Clean and Test Unclean data
# Save cleaned train and test sets


,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,Label
0,3,54,78,104,82,54,3,54,78,82,54,3,4,3,3,3,4,3,4,78,4,3,3,3,3,0
1,4,119,3,3,3,3,78,4,3,3,3,78,78,54,78,78,104,82,119,4,3,78,82,119,4,0
2,119,82,54,3,78,54,78,78,82,4,3,78,82,119,54,3,78,54,78,78,82,4,3,78,82,0
3,153,156,157,122,55,33,5,108,3,3,6,4,55,3,3,4,3,3,5,106,3,6,4,4,4,0
4,54,3,78,54,78,78,82,54,3,78,54,78,78,82,119,4,3,78,82,54,3,4,4,3,4,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
310,5,55,89,89,89,89,89,107,6,107,106,5,55,89,89,89,107,6,107,106,5,55,89,89,89,1
311,50,106,5,90,6,5,5,3,90,90,90,90,6,6,125,5,5,3,90,90,90,90,6,6,125,1
312,57,126,54,126,55,67,54,106,13,5,108,45,3,6,106,5,108,3,6,45,45,54,55,67,13,1
313,55,89,89,89,89,89,89,89,89,89,89,107,6,107,106,5,55,89,89,89,89,89,89,89,89,1


In [67]:
x_train.to_csv('train.csv', index=False)
test_clean.to_csv('test_clean.csv', index=False)
test_unclean.to_csv('test_unclean.csv', index=False)